In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb
from sklearn import metrics as m
from thundersvm import SVC as svmgpu
import calculateWeightUsingGa2 as aresult
import pandas as pd
import itertools
import random
from sklearn.ensemble import VotingClassifier

randomseed = 42
np.random.seed(randomseed)

<IPython.core.display.Javascript object>

1. Read Dataset 
===

In [3]:
xtest = np.array(pd.read_csv("../dataset/xtest.txt"))
xtrain = np.array(pd.read_csv("../dataset/xtrain.txt"))
ytest_original = np.array(pd.read_csv("../dataset/ytest.txt")).ravel()
ytrain_original = np.array(pd.read_csv("../dataset/ytrain.txt")).ravel()

ytrain = ytrain_original.copy()
ytest = ytest_original.copy()


real_xtest=xtest
real_ytest=ytest_original

x=xtrain
y=ytrain_original


<IPython.core.display.Javascript object>

In [4]:
original_scores = []
trial1_scores = [] 

from sklearn.model_selection import KFold

kf = KFold(n_splits=10, random_state=randomseed, shuffle=True)
kf.get_n_splits(x)

print(kf)

data_cv = []

for train_index, test_index in kf.split(x):
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    data_cv.append([[X_train, X_test], [y_train, y_test]])


for i in range(len(data_cv)):
    print("************************** ==> ", i)
    
    xtrain = data_cv[i][0][0]
    ytrain_original = data_cv[i][1][0]    
    xtest = data_cv[i][0][1]
    ytest_original = data_cv[i][1][1]    
    
    ytrain=ytrain_original.copy()
    ytest=ytest_original.copy() 
    
    # member values
    clf = []
    acc = []
    finalacc = []
    ypredproba_all = []
    ypredconfprob_all = []
    
    # orginal score using random forest classifier
    rf = RandomForestClassifier(random_state=randomseed, n_estimators=10)
    rf.fit(xtrain, ytrain)
    print("original score", m.f1_score(ytest, rf.predict(xtest), average="weighted"))
    original_scores.append(m.f1_score(ytest, rf.predict(xtest), average="weighted"))
    
    
    # generate three base classifers using RF,svm and XGBoost

    #================================================= 

    rf=RandomForestClassifier(random_state=randomseed, n_estimators=10)
    rf.fit(xtrain,ytrain)
    rfpred=rf.predict(xtest)
    print('RF f1_score',m.f1_score(ytest,rfpred,average='weighted'))

    clf.append(rf)
    acc.append(m.f1_score(ytest,rfpred,average='weighted'))
    ypredproba_all.append(rf.predict_proba(xtest))

    confmat=m.confusion_matrix(ytest,rfpred)
    confsumh=np.sum(confmat,axis=1)
    propconfmat=confmat.copy()
    for i in range(propconfmat.shape[0]):
        propconfmat[i]= 100*propconfmat[i]/confsumh[i] 
    ypredconfprob_all.append(propconfmat/100)

    #=================================================
    svc=svmgpu(random_state=randomseed,probability=True,C=100,gamma=0.0001)
    svc.fit(xtrain,ytrain)

    svcpred=svc.predict(xtest)
    print('RF svc',m.f1_score(ytest,svcpred,average='weighted'))

    clf.append(svc)
    acc.append(m.f1_score(ytest,svcpred,average='weighted'))
    ypredproba_all.append(svc.predict_proba(xtest))

    confmat=m.confusion_matrix(ytest,svcpred)
    confsumh=np.sum(confmat,axis=1)
    propconfmat=confmat.copy()
    for i in range(propconfmat.shape[0]):
        propconfmat[i]= 100*propconfmat[i]/confsumh[i] 
    ypredconfprob_all.append(propconfmat/100)

    #=================================================
    xgbc=xgb.XGBClassifier(random_state=randomseed,n_estimators=100)
    xgbc.fit(xtrain,ytrain)

    xgbpred=xgbc.predict(xtest)
    print('RF xgbc',m.f1_score(ytest,xgbpred,average='weighted'))


    clf.append(xgbc)
    acc.append(m.f1_score(ytest,xgbpred,average='weighted'))
    ypredproba_all.append(xgbc.predict_proba(xtest))

    confmat=m.confusion_matrix(ytest,xgbpred)
    confsumh=np.sum(confmat,axis=1)
    propconfmat=confmat.copy()
    for i in range(propconfmat.shape[0]):
        propconfmat[i]= 100*propconfmat[i]/confsumh[i] 
    ypredconfprob_all.append(propconfmat/100)

    # =================================================
    # =================================================
    # generate combinations of features 12,7
    comb = list(itertools.combinations(np.arange(0, 12, 1), 7))

    # generate 10 random numbers
    randnums = []
    for i in range(10):
        randnums.append(random.randrange(0, len(comb)))

    print(randnums)

    comb = np.array(comb)[randnums, :]


    for i in range(len(comb)):
        print(i, " ==================== ", comb[i])

        rf = RandomForestClassifier(random_state=randomseed, n_estimators=50)
        rf.fit(xtrain[:, comb[i]], ytrain)
        rfpred = rf.predict(xtest[:, comb[i]])
        print(m.f1_score(ytest, rfpred, average="weighted"))

        clf.append(rf)
        acc.append(m.f1_score(ytest, rfpred, average="weighted"))
        ypredproba_all.append(rf.predict_proba(xtest[:, comb[i]]))

        confmat = m.confusion_matrix(ytest, rfpred)
        confsumh = np.sum(confmat, axis=0)
        propconfmat = confmat.copy()
        for i in range(propconfmat.shape[0]):
            propconfmat[:, i] = 100 * propconfmat[:, i] / confsumh[i]
        ypredconfprob_all.append(propconfmat / 100)

        xgbmodel = xgb.XGBClassifier(random_state=randomseed, n_estimators=50)
        xgbmodel.fit(xtrain, ytrain)
        xgbmodelpred = xgbmodel.predict(xtest)
        print(m.f1_score(ytest, xgbmodelpred, average="weighted"))

        clf.append(xgbmodel)
        acc.append(m.f1_score(ytest, xgbmodelpred, average="weighted"))
        ypredproba_all.append(xgbmodel.predict_proba(xtest))

        confmat = m.confusion_matrix(ytest, xgbmodelpred)
        confsumh = np.sum(confmat, axis=0)
        propconfmat = confmat.copy()
        for i in range(propconfmat.shape[0]):
            propconfmat[:, i] = 100 * propconfmat[:, i] / confsumh[i]
        ypredconfprob_all.append(propconfmat / 100)

    # #=================================================
    #Compute the weight using ga and compute the ensemble accuracy
    import calculateWeightUsingGa2 as aresult
    weightvalga=aresult.getbestvalues(acc)

    finalval=0
    for i in range(len(acc)):
        finalval += weightvalga[i]*ypredproba_all[i]

    print('f1_score',m.f1_score(ytest,np.argmax(finalval,axis=1),average='weighted'))
    print('accuracy_score',m.accuracy_score(ytest,np.argmax(finalval,axis=1)))
    trial1_scores.append(m.f1_score(ytest,np.argmax(finalval,axis=1),average='weighted'))




KFold(n_splits=10, random_state=42, shuffle=True)
************************** ==>  0
original score 0.7637048249937456
RF f1_score 0.7637048249937456
RF svc 0.6550152455835335
RF xgbc 0.7419663394836973
[353, 513, 545, 259, 415, 15, 142, 527, 202, 328]
0  ====================  [ 0  2  4  7  8  9 11]
0.7686563683017452
0.7313201718096775
1  ====================  [ 1  2  3  5  8  9 10]
0.7246944401010287
0.7313201718096775
2  ====================  [ 1  2  4  5  7  9 10]
0.7245835813458326
0.7313201718096775
3  ====================  [ 0  2  3  4  5  7 10]
0.7748883494392745
0.7313201718096775
4  ====================  [ 0  3  5  6  7  8 11]
0.777263708808196
0.7313201718096775
5  ====================  [0 1 2 3 4 8 9]
0.7766296966358786
0.7313201718096775
6  ====================  [0 1 3 4 6 7 9]
0.7692688031143226
0.7313201718096775
7  ====================  [ 1  2  3  7  8  9 10]
0.7174445156361953
0.7313201718096775
8  ====================  [ 0  1  4  5  6  8 11]
0.7812657431696014
0.731320

original score 0.7824638672887874
RF f1_score 0.7824638672887874
RF svc 0.6600698175043455
RF xgbc 0.7433289474536209
[669, 234, 23, 122, 685, 363, 278, 622, 272, 59]
0  ====================  [ 1  5  6  8  9 10 11]
0.7400548928953233
0.7368120611758339
1  ====================  [ 0  1  5  6  7  9 10]
0.7689014005588992
0.7368120611758339
2  ====================  [0 1 2 3 5 6 9]
0.7845382334208353
0.7368120611758339
3  ====================  [ 0  1  2  7  8  9 11]
0.7826072718472763
0.7368120611758339
4  ====================  [ 2  3  4  5  7  9 10]
0.7064472553715074
0.7368120611758339
5  ====================  [ 0  2  5  6  8  9 10]
0.7737768449545857
0.7368120611758339
6  ====================  [ 0  2  3  4  7  8 10]
0.7785879728738868
0.7368120611758339
7  ====================  [ 1  3  4  8  9 10 11]
0.7334134565299625
0.7368120611758339
8  ====================  [ 0  2  3  4  6  8 10]
0.7732735041014894
0.7368120611758339
9  ====================  [ 0  1  2  4  5  6 10]
0.7863503343771611

<IPython.core.display.Javascript object>

In [7]:
print("original socre ", np.mean(original_scores), " std ", np.std(original_scores))
print("ga socre ", np.mean(trial1_scores), " std ", np.std(trial1_scores))

original socre  0.7797252333191481  std  0.006958919713906091
ga socre  0.7874357448780124  std  0.005785704830779771


<IPython.core.display.Javascript object>

4. voting classifier
===

In [6]:
ensemb_clf = []
for i in range(len(clf)):
    ensemb_clf.append([str(type(clf[i])) + str(i), clf[i]])

eclf3 = VotingClassifier(estimators=ensemb_clf, voting="soft", flatten_transform=True)

from sklearn.model_selection import cross_val_score
corssvals=cross_val_score(eclf3, x, y, cv=10)
print("corssvals ", np.mean(corssvals), " std ", np.std(corssvals))

<IPython.core.display.Javascript object>